In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import sys
import os
repo_dir = '/media/user/repos/bulk-euclid-cutouts'
os.path.isdir(repo_dir)
sys.path.insert(0,repo_dir)

In [36]:
import logging
import warnings
import os

from omegaconf import OmegaConf
import pandas as pd
from sklearn.neighbors import KDTree
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
import astropy.units as u
from astropy.table import Table
from astropy.io.fits.verify import VerifyWarning

from bulk_euclid.utils import pipeline_utils
from bulk_euclid.external_targets import pipeline

In [37]:
from astroquery.esa.euclid.core import Euclid
Euclid.login()

INFO: Login to Euclid TAP server [astroquery.esa.euclid.core]


User:  mwalms01
Password:  ········


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid data service [astroquery.esa.euclid.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid cutout service [astroquery.esa.euclid.core]
INFO: OK [astroquery.utils.tap.core]


In [64]:
cfg_dict = dict(
    # TODO change this path
    base_dir='/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version',
    name='external_targets_debug',
    release_name='F-006',
    bands=['VIS', 'NIR_Y'],
    auxillary_products=['MERPSF','MERRMS','MERBKG'],
    
    # these will be filled in automatically using on base_dir
    download_dir=None,
    tile_dir=None,
    cutout_dir=None,
    fits_dir=None,
    sanity_dir=None
    
)
cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc

In [65]:
# !ls /media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version

In [66]:
pipeline.create_folders(cfg)  # now they are filled in

{'base_dir': '/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version', 'name': 'external_targets_debug', 'release_name': 'F-006', 'bands': ['VIS', 'NIR_Y'], 'auxillary_products': ['MERPSF', 'MERRMS', 'MERBKG'], 'download_dir': '/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug', 'tile_dir': '/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug/tiles', 'cutout_dir': '/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug/cutouts', 'fits_dir': '/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug/cutouts/fits', 'sanity_dir': '/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug/sanity'}

In [39]:
external_targets = pd.read_csv('/media/user/repos/bulk-euclid-cutouts/tests/data/102021539_mer_catalog.csv')
external_targets.head()

,id_str,target_ra,target_dec,target_field_of_view
0,a,92.837184,-47.886892,100
1,b,92.836691,-48.111763,100
2,c,92.835639,-48.187139,100
3,d,92.835561,-47.978437,100
4,e,92.835173,-48.105453,100


In [43]:
targets_with_tiles = pipeline.get_matching_tiles(cfg, external_targets)
targets_with_tiles

INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
Found 1957  query results


,tile_index,id_str,target_ra,target_dec,target_field_of_view
0,102021539,a,92.837184,-47.886892,100
1,102021539,b,92.836691,-48.111763,100
3,102021539,d,92.835561,-47.978437,100
4,102021539,e,92.835173,-48.105453,100
5,102021539,f,92.834398,-47.999302,100


In [70]:
# targets_with_tiles.to_dict(orient='records')

[{'tile_index': 102021539,
  'id_str': 'a',
  'target_ra': 92.8371844681179,
  'target_dec': -47.8868920683699,
  'target_field_of_view': 100},
 {'tile_index': 102021539,
  'id_str': 'b',
  'target_ra': 92.836691187603,
  'target_dec': -48.1117628801209,
  'target_field_of_view': 100},
 {'tile_index': 102021539,
  'id_str': 'd',
  'target_ra': 92.835560566775,
  'target_dec': -47.978437467137,
  'target_field_of_view': 100},
 {'tile_index': 102021539,
  'id_str': 'e',
  'target_ra': 92.8351734327103,
  'target_dec': -48.1054532354171,
  'target_field_of_view': 100},
 {'tile_index': 102021539,
  'id_str': 'f',
  'target_ra': 92.8343976295701,
  'target_dec': -47.9993017111107,
  'target_field_of_view': 100}]

In [68]:
pipeline.make_cutouts(cfg, targets_with_tiles)

INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
Found 2  query results
